In [3]:
#from konlpy.tag import Twitter, Kkma
from gensim.models.word2vec import Word2Vec
from copy import deepcopy
import re
import os
from datetime import datetime
import csv
import sys 

#t=Twitter()

TXT_PATH='./korea_txt/'
PDF_PATH='./korea_pdf/'
SAVE_PATH='./korea_wv/'

In [3]:
def pdf_to_text(path):    #pdf 파일을 txt 파일로
    pdf_list=[]
    pdfs=os.listdir(PDF_PATH)
    for pdf in pdfs:
        ext = os.path.splitext(pdf)[-1]
        name=os.path.splitext(pdf)[-2]+'.txt'
        
        if ext=='.pdf':
            #pdf_list.append("%s%s" % (path, pdf))
            #pdf_name.append("%s"%pdf)
            os.system("python pdfminer.six-20170720/tools/pdf2txt.py -o %s%s %s%s" %(TXT_PATH, name, PDF_PATH, pdf))

def get_file(path): #txt파일 불러오기
    filelist=[]
    files = os.listdir(TXT_PATH) # txt파일에 있는 파일 리스트를 가져옴
    for file in files:
        ext = os.path.splitext(file)[-1]
            
        if ext == '.txt':
            filelist.append("%s%s" % (path, file))
    
    content=[]
    for file in filelist:
        content.append(preprocessing(open_file(file))) #content리스트 하나에 하나의 논문
    
    sentences = [tokenize(d) for d in content] #논문 하나가 한 리스트에 토큰화
    return sentences

def open_file(file): #txt 파일 오픈
    txt_file = open(file, encoding='utf-8')
    content = txt_file.read()
    txt_file.close()
    return content

def preprocessing(content): #전처리
    content = re.sub('\\xa0', '', content)
    content = re.sub('\\n', '', content)
    content = re.sub('\\\\xa0', '', content)
    content = re.sub('\\\\n', '', content)
    content = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@#$%&\\\=\(\'\")]', '', content)
    content = ' '.join(content.split())
                     
    return content

def tokenize(doc): #토크나이징
    return ['/'.join(t) for t in t.pos(doc, norm=True, stem=True)]

def wv_update(file): #새로운 논문 업데이트
    name=os.path.splitext(file)[-2]+'.txt'
    os.system("python pdfminer.six-20170720/tools/pdf2txt.py -o %s%s %s%s" %(TXT_PATH, name, PDF_PATH, file))
    
    new_token=[]
    content.append(preprocessing(open_file(TXT_PATH + name)))
    
    
    new_token=[tokenize(d) for d in content]
    
    wv_model_ko.build_vocab(new_token, update=True)
    wv_model_ko.train(new_token, total_examples=wv_model_ko.corpus_count, epochs=wv_model_ko.epochs)
    wv_model_ko.init_sims(replace=True)
    now=datetime.now()
    wv_model_ko.save('%s%s%s%s%s%s' % (SAVE_PATH ,now.year, now.month, now.day, now.hour, now.minute))

In [4]:
%%time
pdf_to_text(PDF_PATH) #pdf 파일 txt로 변환 실행

Wall time: 16min 20s


In [5]:
%%time
sentences=get_file(TXT_PATH) #txt파일 read 후 토큰화

Wall time: 1min 21s


In [6]:
sentences

[['1930/Number',
  '년대/Noun',
  '장편소설/Noun',
  '에/Josa',
  '나타나다/Verb',
  '고향/Noun',
  '의식/Noun',
  '이기영/Noun',
  '『/Foreign',
  '고향/Noun',
  '』/Foreign',
  '강경애/Noun',
  '『/Foreign',
  '인간/Noun',
  '문제/Noun',
  '』/Foreign',
  '한설야/Noun',
  '『/Foreign',
  '황혼/Noun',
  '』/Foreign',
  '을/Josa',
  '중심/Noun',
  '으로/Josa',
  'A/Alpha',
  'Study/Alpha',
  'on/Alpha',
  'the/Alpha',
  'Consciousness/Alpha',
  'of/Alpha',
  'Home/Alpha',
  'in/Alpha',
  'the/Alpha',
  'Novel/Alpha',
  'of/Alpha',
  'the/Alpha',
  '1930/Number',
  's/Alpha',
  'In/Alpha',
  'the/Alpha',
  'Lee/Alpha',
  'Ki/Alpha',
  'Young/Alpha',
  'KOHYANG/Alpha',
  '고향/Noun',
  'Gang/Alpha',
  'Keong/Alpha',
  'Ae/Alpha',
  'INKAN/Alpha',
  'MOONJE/Alpha',
  '인간/Noun',
  '문제/Noun',
  'Han/Alpha',
  'Seol/Alpha',
  'Ya/Alpha',
  'HWANGHON/Alpha',
  '황혼/Noun',
  '저자/Noun',
  'Authors/Alpha',
  '출처/Noun',
  'Source/Alpha',
  '발행/Noun',
  '처/Noun',
  'PublisherURLAPA/Alpha',
  'Style/Alpha',
  '이용/Noun',
  '정보/Noun',
  'Accesse

In [7]:
len(sentences)

120

In [39]:
%%time
wv_model_ko = Word2Vec(sentences, size=100, window = 8, min_count=10, workers=4, iter=100, sg=1)
#100차원 벡터, 주변단어 2개 참조, 출연빈도 20번이상, CPU 쿼드코어, 100번 반복, CBOW, Skip-Gram중 후자
now=datetime.now()
wv_model_ko.save('%s%s%s%s%s%s' % (SAVE_PATH, now.year, now.month, now.day, now.hour, now.minute))

Wall time: 7min 39s


In [47]:
wv_model_ko.wv[tokenize('사전')]

array([[-0.05525103,  0.242103  , -0.22075434,  0.44539022,  0.42720997,
        -0.47559723,  0.11801864,  0.4862534 , -0.15505666,  0.3057172 ,
         0.7320078 , -0.5674984 ,  0.24719673,  0.4504992 , -0.38204843,
         0.02575449,  0.12919456,  0.20583297,  0.24186315,  0.19419189,
        -0.25077114,  0.74359924, -0.22914341,  0.4337914 , -0.3601756 ,
        -0.52490026,  0.1824163 , -0.37821186, -0.03428338, -0.37663296,
        -0.31988767, -0.13137777, -0.42600092, -0.36438775, -0.24405944,
         0.00295244,  0.459734  ,  0.12913296, -0.2768928 , -0.42211902,
         0.78342265, -0.24784285, -0.03100568, -0.24683602,  0.3354473 ,
         0.14509888, -0.87685406, -0.36918423,  0.02687013, -0.4739646 ,
        -0.18716468,  0.17470062, -0.11831488,  0.0905275 ,  0.04786654,
        -0.07267198,  0.37439436,  0.02525811,  0.03344254, -0.7169699 ,
        -0.11279815, -0.58096075, -0.17930289,  0.3167087 , -0.25873917,
        -0.00269555, -0.4082    ,  0.12165979,  0.1

In [40]:
copy_model=Word2Vec.load(SAVE_PATH+'20184161555')

In [44]:
copy_model.wv.most_similar(tokenize('엄마'))

[('소녀/Noun', 0.575179934501648),
 ('밀/Noun', 0.5747277140617371),
 ('아빠/Noun', 0.5268253087997437),
 ('오빠/Noun', 0.5148156881332397),
 ('죄책감/Noun', 0.5147483944892883),
 ('토끼/Noun', 0.5097417831420898),
 ('손/Noun', 0.5052055716514587),
 ('아기/Noun', 0.4793655276298523),
 ('태우다/Verb', 0.47653013467788696),
 ('그날/Noun', 0.46371597051620483)]

In [42]:
wv_model_ko.corpus_count

120

In [41]:
len(wv_model_ko.wv.vocab)

7714

In [43]:
wv_model_ko.wv.most_similar(tokenize('엄마'))

[('소녀/Noun', 0.575179934501648),
 ('밀/Noun', 0.5747277140617371),
 ('아빠/Noun', 0.5268253087997437),
 ('오빠/Noun', 0.5148156881332397),
 ('죄책감/Noun', 0.5147483944892883),
 ('토끼/Noun', 0.5097417831420898),
 ('손/Noun', 0.5052055716514587),
 ('아기/Noun', 0.4793655276298523),
 ('태우다/Verb', 0.47653013467788696),
 ('그날/Noun', 0.46371597051620483)]

In [45]:
wv_model_ko.wv.similarity(*tokenize('소설 명사'))

0.09330516288086523

In [92]:
wv_model_ko.predict_output_word(tokenize('프로테오믹스'))

[('XML/Alpha', 0.00029627155),
 ('문서/Noun', 0.00028014262),
 ('컴퓨터/Noun', 0.0002618768),
 ('미국/Noun', 0.00026059002),
 ('전문/Noun', 0.00025785508),
 ('센터/Noun', 0.00025701264),
 ('전산/Noun', 0.0002515059),
 ('국제/Noun', 0.00025058712),
 ('｢/Foreign', 0.00024986523),
 ('연구소/Noun', 0.0002492874)]

In [30]:
from matplotlib import font_manager, rc
font_fname='c:/windows/fonts/H2GTRM.TTF'
font_name=font_manager.FontProperties(fname=font_fname).get_name()
rc('font', family=font_name)

import pandas as pd
pd.options.mode.chained_assignment = None 
import numpy as np
import nltk


from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline

def tsne_plot(model):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []

    for word in model.wv.vocab:
        tokens.append(model[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()

In [37]:
non=open('./txt/생식생물학에서_프로테오믹스의_응용.txt', encoding='utf-8').read()

content2=[]

content2.append(preprocessing(non)) #content리스트 하나에 하나의 논문

non_ko = [tokenize(d) for d in content2]  #논문 하나가 한 리스트에 토큰화

non_ko

[['생식/Noun',
  '생물학/Noun',
  '에서/Josa',
  '프로/Noun',
  '테오/Noun',
  '믹스/Noun',
  '의/Josa',
  '응용/Noun',
  'Potential/Alpha',
  'Importance/Alpha',
  'of/Alpha',
  'Proteomics/Alpha',
  'in/Alpha',
  'Research/Alpha',
  'of/Alpha',
  'Reproductive/Alpha',
  'Biology/Alpha',
  '저자/Noun',
  'Authors/Alpha',
  '출처/Noun',
  'Source/Alpha',
  '김/Noun',
  '호승/Noun',
  '윤용/Noun',
  '달/Noun',
  'HoSeung/Alpha',
  'Kim/Alpha',
  'YongDal/Alpha',
  'Yoon/Alpha',
  '발생/Noun',
  '과/Josa',
  '생식/Noun',
  '81/Number',
  '20046/Number',
  '19/Number',
  '9/Number',
  'pagesDevelopmant/Alpha',
  'Reproduction/Alpha',
  '81/Number',
  '20046/Number',
  '19/Number',
  '9/Number',
  'pagesPublisher/Alpha',
  '한국/Noun',
  '발생생물학/Noun',
  '회/Noun',
  'The/Alpha',
  'Korea/Alpha',
  'Society/Alpha',
  'Of/Alpha',
  'Developmental/Alpha',
  'BiologyURLhttpwwwdbpiacokrArticleNODE/Alpha',
  '00557235/Number',
  'APA/Alpha',
  'Style/Alpha',
  '김/Noun',
  '호승/Noun',
  '윤용/Noun',
  '달/Noun',
  '2004/Number',
  '생

In [36]:
content2

['생물정보학기반 치쿤구니아 바이러스 항원결정부위의 예측Prediction of Epitope for Chikungunya Virus based on Bioinformatics저자Authors이지후 김학용Jihoo Lee Hak Yong Kim출처Source한국콘텐츠학회 종합학술대회 논문집 201411 5556 2 pages발행처Publisher한국콘텐츠학회The Korea Contents SocietyURLhttpwwwdbpiacokrArticleNODE02500941APA Style이지후 김학용 2014 생물정보학기반 치쿤구니아 바이러스 항원결정부위의 예측 한국콘텐츠학회 종합학술대회논문집 5556이용정보Accessed122417220180411 1938 KST저작권 안내DBpia에서 제공되는 모든 저작물의 저작권은 원저작자에게 있으며 누리미디어는 각 저작물의 내용을 보증하거나 책임을 지지 않습니다 그리고 DBpia에서 제공되는 저작물은 DBpia와 구독계약을 체결한 기관소속 이용자 혹은 해당 저작물의 개별 구매자가 비영리적으로만 이용할 수 있습니다 그러므로 이에 위반하여 DBpia에서 제공되는 저작물을 복제 전송 등의 방법으로 무단 이용하는 경우 관련 법령에 따라 민 형사상의 책임을 질 수 있습니다Copyright InformationCopyright of all literary works provided by DBpia belongs to the copyright holdersand Nurimedia does not guarantee contents of the literary work or assume responsibility for thesame In addition the literary works provided by DBpia may only be used by the users affiliated to the institutions which executed a subscription agreement with DBpia or theindiv